# Final Project

Importing necessary packages

In [1]:
import xpress as xp
import pandas as pd
import numpy as np

#### Defining a problem

In [2]:
prob = xp.problem(name="BikeStationsSimple")

/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69467/3827034396.py:1: LicenseWarning: Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
  prob = xp.problem(name="BikeStationsSimple")


#### Sets

In [3]:
zones_df = pd.read_csv("zone_predictions_filtered_poi_gt_0.csv")

In [4]:
def accessibility_exponential(zones, beta=0.8):
    """
    zones: DataFrame with columns ['zone_id', 'latitude', 'longitude']
    beta:  decay parameter for exp(-beta * distance_km)

    Returns:
        dist_dict  : {(zone_id_i, zone_id_j): distance_km}
        access_dict: {(zone_id_i, zone_id_j): exp(-beta * distance_km)}
    """
    
    # Extract arrays
    zone_ids = zones['zone_id'].values
    lat = np.radians(zones['latitude'].values)
    lon = np.radians(zones['longitude'].values)
    
    # Broadcast
    lat_i = lat[:, None]
    lat_j = lat[None, :]
    lon_i = lon[:, None]
    lon_j = lon[None, :]
    
    # Haversine
    R = 6371.0
    dlat = lat_j - lat_i
    dlon = lon_j - lon_i

    a = (np.sin(dlat/2)**2 +
         np.cos(lat_i) * np.cos(lat_j) * np.sin(dlon/2)**2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    dist_km = R * c
    access = np.exp(-beta * dist_km)

    # Build dictionaries using zone_ids
    dist_dict = {
        (zone_ids[i], zone_ids[j]): float(dist_km[i, j])
        for i in range(len(zone_ids))
        for j in range(len(zone_ids))
    }

    access_dict = {
        (zone_ids[i], zone_ids[j]): float(access[i, j])
        for i in range(len(zone_ids))
        for j in range(len(zone_ids))
    }

    return dist_dict, access_dict

In [5]:
# --- index sets ---
J = zones_df['zone_id']  # candidate stations
I = zones_df['zone_id']  # demand zones

In [6]:
commercial_zone_ids = [
    387, 414, 415, 416, 443, 444, 452, 453,
    481, 482, 483, 510, 511, 512, 538, 539, 540,
    564, 565, 566, 589, 590, 611, 612, 613
]

In [7]:
import pandas as pd

group_files = [f"group_{k}_new.csv" for k in range(1, 11)]

zone_groups = []  # this will become a list of lists: [group1, group2, ... group5]
zone_groups_all = []
for f in group_files:
    df = pd.read_csv(f)
    group = list((set(df["zone_id"].tolist()).intersection(I)))
    zone_groups.append(group)
    zone_groups_all.extend(group)

#### Decision Variables

In [8]:
# x_j: open station j (0/1)
x = {j: xp.var(name=f"x_{j}", vartype=xp.binary) for j in J}

# c_j: docks at station j (integer)
c = {j: xp.var(name=f"c_{j}", vartype=xp.integer, lb=0) for j in J}

# y_ij: trips from zone i served by station j (continuous, ≥0)
y = {(i, j): xp.var(name=f"y_{i}_{j}", vartype=xp.integer, lb=0)
     for i in I for j in J}

# z_k: commercial zone i reach target (0/1)
z = {i: xp.var(name=f"z_{i}", vartype=xp.binary)
     for i in commercial_zone_ids}

prob.addVariable(list(x.values()) +
                 list(c.values()) +
                 list(y.values()) +
                 list(z.values()))

/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69467/3059644690.py:2: DeprecationWarning: Deprecated in Xpress 9.5: create a linked variable by calling problem.addVariable()
  x = {j: xp.var(name=f"x_{j}", vartype=xp.binary) for j in J}
/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69467/3059644690.py:5: DeprecationWarning: Deprecated in Xpress 9.5: create a linked variable by calling problem.addVariable()
  c = {j: xp.var(name=f"c_{j}", vartype=xp.integer, lb=0) for j in J}
/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69467/3059644690.py:8: DeprecationWarning: Deprecated in Xpress 9.5: create a linked variable by calling problem.addVariable()
  y = {(i, j): xp.var(name=f"y_{i}_{j}", vartype=xp.integer, lb=0)
/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69467/3059644690.py:12: DeprecationWarning: Deprecated in Xpress 9.5: create a linked variable by calling problem.addVariable()
  z = {i: xp.var(name=f"z_{i}", vartype=xp.binary)


#### Parameters

In [9]:
# ==============================
#  MODEL PARAMETERS
# ==============================

# --- Financial parameters ---
max_budget = 3_000_000        # total budget [£]
fixed_cost = 6_000            # station setup cost [£ per station]
dock_cost = 4_000             # dock installation cost [£ per dock]

# --- Capacity parameters ---
dock_max = 15                 # maximum docks per station
dock_min = 3                  # minimum docks if opened

# --- Service Rates ---
theta_df = pd.read_csv("zone_service_rate_new.csv")
service_rate = theta_df.set_index("zone_id")["theta"].to_dict()            # trips per dock per day

# --- Stations parameters ---
station_min = 0                # minimum number of stations
station_max = 120              # maximum number of stations

# --- Demand parameters ---
zones_df["predicted_total_demand"] = zones_df["predicted_total_demand"]
demand_per_zone = zones_df.set_index('zone_id')['predicted_total_demand'].to_dict()

# --- Accessibiility parameters ---
distance,accessibility = accessibility_exponential(zones_df)

# --- Spatial spacing (exclude commercial zones) ---
min_space = 0.7                # minimal allowed distance between stations [km]

pairs = []

for zi in I:
    for zj in J:
        if zi >= zj:    # avoid duplicates + self-pairs
            continue

        # skip commercial zones
        if zi in commercial_zone_ids or zj in commercial_zone_ids:
            continue

        # check spacing rule using zone_id pairs
        if distance[(zi, zj)] < min_space:
            pairs.append([zi, zj])
            
# --- Commercial parameters ---
commercial_target = 0.9         # minimal demand target for commercial financial support
subsidy_per_zone = 2000

# --- Zone Groups ---
station_min_zone_group = 4      # minimal number of station for each zone group

#### Objective Function

In [10]:
prob.setObjective(xp.Sum(y[i, j] for i in I for j in J), sense=xp.maximize)

#### Constraints

In [11]:
# (1) Budget
prob.addConstraint(
    xp.Sum(fixed_cost * x[j] + dock_cost * c[j] for j in J) - xp.Sum(subsidy_per_zone * z[i] for i in commercial_zone_ids)<= max_budget)

# (2) Dock linking (upper & minimum when open)
prob.addConstraint([c[j] <= dock_max * x[j] for j in J])
prob.addConstraint([c[j] >= dock_min * x[j] for j in J])

# (3) Demand conservation (per zone)
prob.addConstraint([xp.Sum(y[i, j] for j in J) <= demand_per_zone[i] for i in I])

# (4) Station service capacity (per station)
prob.addConstraint([xp.Sum(y[i, j] for i in I) <= service_rate[j] * c[j] for j in J])

# (5) Spacing within 700 m
prob.addConstraint([x[j] + x[k] <= 1 for (j, k) in pairs])

# (6) Accessibility Constraint
prob.addConstraint(y[i,j] <= accessibility[i,j]*demand_per_zone[i] for i in I for j in J)

# (7) Number of station
prob.addConstraint(xp.Sum(x[j] for j in J) >= station_min)
prob.addConstraint(xp.Sum(x[j] for j in J) <= station_max)

# (8) Commercial District Constraints
prob.addConstraint([xp.Sum(y[i, j] for j in J) >= commercial_target * demand_per_zone[i] * z[i] for i in commercial_zone_ids])

# (9) Zone Group Station Constraints
prob.addConstraint(xp.Sum(x[j] for j in zone_groups[i]) >= station_min_zone_group for i in range(len(zone_groups)))

#### Solve

In [12]:
# write & solve
prob.write("problem", "lp")
xp.setOutputEnabled(True)

# --- SOLVER CONTROLS AND LIMITS ---

# Time limit (in seconds)
prob.setControl('MAXTIME', 300)          # stop after 5 minutes

# Node limit (branch-and-bound iterations)
prob.setControl('MAXNODE', 10000)         # stop after 10000 explored nodes

# MIP gap tolerance (relative)
prob.setControl('MIPRELSTOP', 0.02)      # stop when within 2% of optimality

# Optional: control verbosity (0 = silent, 1 = summary, 2 = full)
prob.setControl('OUTPUTLOG', 1)

# --- SOLVE AND REPORT ---
print("🚀 Starting solver with runtime and gap limits ...")
prob.solve()

print("\n=== SOLVER SUMMARY ===")
print("Status:", prob.getProbStatusString())

# MIP gap (if available)
try:
    print("Relative MIP gap:", prob.getAttrib('miprelgap'))
except:
    print("No MIP gap info available (non-MIP model).")

print("Objective value:", prob.getObjVal())
print("Solver time (sec):", prob.getAttrib('time'))
print("Nodes explored:", prob.getAttrib('nodes'))
prob.solve()

🚀 Starting solver with runtime and gap limits ...
FICO Xpress v9.7.0, Hyper, solve started 0:01:25, Nov 26, 2025
Heap usage: 88MB (peak 88MB, 20MB system)
Maximizing MILP BikeStationsSimple using up to 14 threads and up to 24GB memory, with these control settings:
MAXNODE = 10000
MAXTIME = 300
OUTPUTLOG = 1
MIPRELSTOP = .02
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
    149080 rows       147480 cols       455050 elements    147480 entities
Presolved problem has:
      1881 rows        20382 cols        43635 elements     20362 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 141MB (peak 238MB, 20MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  6.00e+03] / [ 3.12e-02,  1.94e+00]
  RHS and bounds [min,max] : [ 3.06e-07,  3.00e+06] / [ 1.00e+00,  1.88e+02]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscalin

/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69467/1850318658.py:24: DeprecationWarning: Deprecated in Xpress 9.5: use problem.attributes.solvestatus and problem.attributes.solstatus instead
  print("Status:", prob.getProbStatusString())
/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69467/1850318658.py:32: DeprecationWarning: Deprecated in Xpress 9.5: use problem.attributes.objval instead
  print("Objective value:", prob.getObjVal())


a         4378.000000  5597.000000      1                 21.78%       0      0
b         4691.000000  5597.000000      2                 16.19%       0      0
q         4916.000000  5597.000000      3                 12.17%       0      0
k         4966.000000  5597.000000      4                 11.27%       0      0
R         4974.000000  5597.000000      5                 11.13%       0      0
R         4981.000000  5597.000000      6                 11.01%       0      0
R         4988.000000  5597.000000      7                 10.88%       0      0
R         4995.000000  5597.000000      8                 10.76%       0      0
R         5002.000000  5597.000000      9                 10.63%       0      0
R         5008.000000  5597.000000     10                 10.52%       0      0
R         5014.000000  5597.000000     11                 10.42%       0      0
R         5020.000000  5597.000000     12                 10.31%       0      0
R         5026.000000  5597.000000     1

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)

#### Summary

In [18]:
import numpy as np          # numerical helpers
import pandas as pd         # tables / DataFrames

# ==============================================================
# 1) EXTRACT DECISION VARIABLES INTO DATAFRAMES
# ==============================================================

# --- 1A) x_j: open station j (0/1) --------------------------------
x_keys = list(x.keys())                                        # station indices j
x_vals = prob.getSolution([x[j] for j in x_keys])              # solution values

open_df = (
    pd.DataFrame({
        "zone_id": x_keys,
        "open":   x_vals
    })
    .set_index("zone_id")                                      # index by zone_id
)

# --- 1B) c_j: docks at station j -------------------------------
c_keys = list(c.keys())                                        # station indices j
c_vals = prob.getSolution([c[j] for j in c_keys])              # number of docks

docks_df = (
    pd.DataFrame({
        "zone_id": c_keys,
        "docks":   c_vals
    })
    .set_index("zone_id")
)

# --- 1C) y_ij: trips from zone i served by station j ----------
y_keys = list(y.keys())                                        # list of (i, j) tuples
y_vals = prob.getSolution([y[k] for k in y_keys])              # trips for each (i,j)

# build index maps so we can fill a dense matrix
i_index = {i: idx for idx, i in enumerate(I)}                  # map zone i → row
j_index = {j: idx for idx, j in enumerate(J)}                  # map station j → col

Y = np.zeros((len(I), len(J)))                                 # dense matrix for y_ij

for val, (i, j) in zip(y_vals, y_keys):
    Y[i_index[i], j_index[j]] = val

y_df = pd.DataFrame(Y, index=I, columns=J)                     # rows = zones, cols = stations

# ==============================================================
# 2) AGGREGATIONS PER STATION / PER ZONE
# ==============================================================

# total trips originating from each station j (sum over i)
trips_from_stations = y_df.sum(axis=0).to_frame("trips_from_stations")   # index = J

# total trips served in each zone i (sum over j)
trips_serving_zones = y_df.sum(axis=1).to_frame("trips_serving_zones")   # index = I

# demand per zone (input dict → DataFrame indexed by zone_id)
demand_df = pd.DataFrame.from_dict(
    demand_per_zone,
    orient="index",
    columns=["Demand"]
)

# ==============================================================
# 3) BUILD MASTER SOLUTION DATAFRAME `sol`
# ==============================================================

sol = (
    zones_df[["zone_id"]]                                      # all candidate zones
    .merge(open_df,             left_on="zone_id", right_index=True, how="left")
    .merge(docks_df,            left_on="zone_id", right_index=True, how="left")
    .merge(trips_from_stations, left_on="zone_id", right_index=True, how="left")
    .merge(trips_serving_zones, left_on="zone_id", right_index=True, how="left")
    .merge(demand_df,           left_on="zone_id", right_index=True, how="left")
)

# service rate per zone = fraction of demand that is served
sol["Rate"] = sol["trips_serving_zones"] / sol["Demand"]

# ==============================================================
# 4) BASIC STATION / DOCK / COVERAGE STATISTICS
# ==============================================================

# colour codes for pretty terminal printing
BLUE = "\033[94m"
BOLD = "\033[1m"
END  = "\033[0m"

# open stations subset
sol_open = sol[sol["open"] == 1]

# masks for zone types
commercial_mask = sol["zone_id"].isin(commercial_zone_ids)     # commercial zones
peripheral_mask = sol["zone_id"].isin(zone_groups_all)         # peripheral zones

# --- station counts --------------------------------------------
n_stations_all        = len(sol_open)
n_stations_commercial = sol_open["zone_id"].isin(commercial_zone_ids).sum()
n_stations_peripheral = sol_open["zone_id"].isin(zone_groups_all).sum()

# --- dock statistics -------------------------------------------
min_docks    = sol_open["docks"].min()
avg_docks    = sol_open["docks"].mean()
median_docks = sol_open["docks"].median()
max_docks    = sol_open["docks"].max()
std_docks    = sol_open["docks"].std()
total_docks  = sol_open["docks"].sum()

# --- demand-weighted coverage (ALL / COMMERCIAL / PERIPHERAL) --
avg_rate_all = (
    sol["trips_serving_zones"].sum() / sol["Demand"].sum()
) * 100

avg_rate_commercial = (
    sol.loc[commercial_mask, "trips_serving_zones"].sum() /
    sol.loc[commercial_mask, "Demand"].sum()
) * 100

avg_rate_peripheral = (
    sol.loc[peripheral_mask, "trips_serving_zones"].sum() /
    sol.loc[peripheral_mask, "Demand"].sum()
) * 100

# zones that are covered at all (served > 0 and not overserved)
n_zones_covered = len(sol[(sol["Rate"] <= 1) & (sol["Rate"] > 0)])

# objective value = total trips (as in model)
obj_trips = int(prob.attributes.objval)

# ==============================================================
# 5) SPATIAL ACCESSIBILITY: NEAREST-NEIGHBOUR DISTANCES
# ==============================================================

# open stations with their coordinates
stations = sol_open.merge(
    zones_df[["zone_id", "latitude", "longitude"]],
    on="zone_id",
    how="left"
)

def avg_nearest_neighbor_distance(df):
    """
    Compute average distance (km) from each station to its nearest other station.
    Uses haversine distance on latitude/longitude.
    """
    n = len(df)
    if n < 2:
        return np.nan

    # convert to radians
    lat = np.radians(df["latitude"].values)
    lon = np.radians(df["longitude"].values)

    # pairwise differences using broadcasting
    lat_i = lat[:, None]
    lat_j = lat[None, :]
    lon_i = lon[:, None]
    lon_j = lon[None, :]

    # haversine formula
    R = 6371.0  # Earth radius in km
    dlat = lat_j - lat_i
    dlon = lon_j - lon_i
    a = np.sin(dlat / 2) ** 2 + np.cos(lat_i) * np.cos(lat_j) * np.sin(dlon / 2) ** 2
    dist = 2 * R * np.arcsin(np.sqrt(a))

    # ignore self-distances on diagonal
    np.fill_diagonal(dist, np.inf)

    # nearest neighbour for each station, then mean
    nearest = dist.min(axis=1)
    return float(nearest.mean())

# masks on the station set
stations_commercial = stations[stations["zone_id"].isin(commercial_zone_ids)]
stations_peripheral = stations[stations["zone_id"].isin(zone_groups_all)]

# average nearest-neighbour distances (km)
avg_nn_all        = avg_nearest_neighbor_distance(stations)
avg_nn_commercial = avg_nearest_neighbor_distance(stations_commercial)
avg_nn_peripheral = avg_nearest_neighbor_distance(stations_peripheral)

def km_to_times(d):
    """
    Convert km into walking and cycling time (minutes).
    Assumes ~5 km/h walking and ~15 km/h cycling.
    """
    if np.isnan(d):
        return np.nan, np.nan
    walk_min = d * 12   # 60 / 5
    bike_min = d * 4    # 60 / 15
    return walk_min, bike_min

walk_all,  bike_all  = km_to_times(avg_nn_all)
walk_comm, bike_comm = km_to_times(avg_nn_commercial)
walk_peri, bike_peri = km_to_times(avg_nn_peripheral)

# ==============================================================
# 6) PRINT ALL KEY RESULTS (ONE CELL OUTPUT)
# ==============================================================

# --- station counts & docks ------------------------------------
print(f"Number of stations opened (all zones): {BLUE}{BOLD}{n_stations_all}{END}")
print(f"Number of stations in commercial zones: {BLUE}{BOLD}{n_stations_commercial}{END}")
print(f"Number of stations in peripheral zones: {BLUE}{BOLD}{n_stations_peripheral}{END}")

print(f"\nMin number of docks per open station: {BLUE}{BOLD}{min_docks:.0f}{END} docks")
print(f"Avg number of docks per open station: {BLUE}{BOLD}{avg_docks:.1f}{END} docks")
print(f"Median number of docks per open station: {BLUE}{BOLD}{median_docks:.1f}{END} docks")
print(f"Max number of docks per open station: {BLUE}{BOLD}{max_docks:.0f}{END} docks")
print(f"Std of docks per open station: {BLUE}{BOLD}{std_docks:.1f}{END} docks")
print(f"Total number of docks: {BLUE}{BOLD}{total_docks:.0f}{END} docks")

# --- demand coverage -------------------------------------------
print(f"\nAverage demand satisfied (all zones): "
      f"{BLUE}{BOLD}{avg_rate_all:.1f}%{END}")
print(f"Average demand satisfied in commercial zones: "
      f"{BLUE}{BOLD}{avg_rate_commercial:.1f}%{END}")
print(f"Average demand satisfied in peripheral zones: "
      f"{BLUE}{BOLD}{avg_rate_peripheral:.1f}%{END}")

print(f"\nNumber of zones covered: {BLUE}{BOLD}{n_zones_covered}{END}")
print(f"Demand coverage rate (all zones): "
      f"{BLUE}{BOLD}{avg_rate_all:.1f}%{END}")   # same as avg_rate_all

print(f"Number of trips (objective function value): "
      f"{BLUE}{BOLD}{obj_trips}{END} trips")

# --- spatial accessibility (nearest neighbour distances) -------
print(f"\nAvg distance to nearest open station (ALL): "
      f"{BLUE}{BOLD}{avg_nn_all:.2f}{END} km "
      f"(≈{walk_all:.0f} min walking, ≈{bike_all:.0f} min by bike)")

print(f"Avg distance to nearest open station (COMMERCIAL): "
      f"{BLUE}{BOLD}{avg_nn_commercial:.2f}{END} km "
      f"(≈{walk_comm:.0f} min walking, ≈{bike_comm:.0f} min by bike)")

print(f"Avg distance to nearest open station (PERIPHERAL): "
      f"{BLUE}{BOLD}{avg_nn_peripheral:.2f}{END} km "
      f"(≈{walk_peri:.0f} min walking, ≈{bike_peri:.0f} min by bike)")

Number of stations opened (all zones): 85
Number of stations in commercial zones: 9
Number of stations in peripheral zones: 59

Min number of docks per open station: 3 docks
Avg number of docks per open station: 7.5 docks
Median number of docks per open station: 6.0 docks
Max number of docks per open station: 15 docks
Std of docks per open station: 4.5 docks
Total number of docks: 634 docks

Average demand satisfied (all zones): 98.8%
Average demand satisfied in commercial zones: 100.0%
Average demand satisfied in peripheral zones: 97.6%

Number of zones covered: 383
Demand coverage rate (all zones): 98.8%
Number of trips (objective function value): 5532 trips

Avg distance to nearest open station (ALL): 1.21 km (≈15 min walking, ≈5 min by bike)
Avg distance to nearest open station (COMMERCIAL): 0.92 km (≈11 min walking, ≈4 min by bike)
Avg distance to nearest open station (PERIPHERAL): 1.33 km (≈16 min walking, ≈5 min by bike)


#### Plot

In [16]:
import folium
import numpy as np
import pandas as pd

# ---------------------------------
# 1. Prepare data: open stations + coordinates
# ---------------------------------
stations = (
    sol[sol["open"] == 1]
    .merge(
        zones_df[["zone_id", "latitude", "longitude"]],
        on="zone_id",
        how="left"
    )
    .copy()
)

# classify station type
stations["type"] = "Other"
stations.loc[stations["zone_id"].isin(zone_groups_all), "type"] = "Peripheral"
stations.loc[stations["zone_id"].isin(commercial_zone_ids), "type"] = "Commercial"

# ---------------------------------
# 2. Map centre
# ---------------------------------
center_lat = stations["latitude"].mean()
center_lon = stations["longitude"].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles="CartoDB.Voyager"
)

# ---------------------------------
# 3. Style settings
# ---------------------------------
color_map = {
    "Commercial": "#e41a1c",  # red
    "Peripheral": "#377eb8",  # blue
    "Other":      "#4daf4a"   # green
}

# scale radius a bit so sizes look reasonable
min_radius = 4
max_radius = 18
d_min = stations["docks"].min()
d_max = stations["docks"].max()

def dock_to_radius(d):
    if d_max == d_min:
        return (min_radius + max_radius) / 2
    return min_radius + (d - d_min) * (max_radius - min_radius) / (d_max - d_min)

# ---------------------------------
# 4. Add station circles
# ---------------------------------
for _, row in stations.iterrows():
    radius = dock_to_radius(row["docks"])
    stype  = row["type"]
    
    popup_html = (
        f"<b>Zone ID:</b> {row['zone_id']}<br>"
        f"<b>Type:</b> {stype}<br>"
        f"<b>Docks:</b> {int(row['docks'])}"
    )
    
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=radius,
        color=color_map[stype],
        fill=True,
        fill_color=color_map[stype],
        fill_opacity=0.8,
        weight=1,
        popup=folium.Popup(popup_html, max_width=250),
    ).add_to(m)

# ---------------------------------
# 5. Add a simple legend
# ---------------------------------
legend_html = """
<div style="
    position: fixed;
    top: 20px;
    right: 300px;   /* <-- move legend left by increasing this value */
    z-index: 9999;
    background-color: white;
    padding: 12px 16px;
    border: 1px solid #ccc;
    border-radius: 6px;
    font-size: 13px;
    line-height: 1.4;
    box-shadow: 0 2px 6px rgba(0,0,0,0.2);
">
<b style="font-size:14px;">Station type</b><br>
<div style="margin-top:6px;">
    <span style="display:inline-block;width:12px;height:12px;background:#e41a1c;
                 border-radius:2px;margin-right:6px;"></span>
    Commercial Area
</div>
<div style="margin-top:4px;">
    <span style="display:inline-block;width:12px;height:12px;background:#377eb8;
                 border-radius:2px;margin-right:6px;"></span>
    Peripheral Area
</div>
<div style="margin-top:4px;">
    <span style="display:inline-block;width:12px;height:12px;background:#4daf4a;
                 border-radius:2px;margin-right:6px;"></span>
    Other Area
</div>
<div style="margin-top:10px;font-size:12px;color:#444;">
    Circle size shows number of docks.
</div>
</div>
"""



m.get_root().html.add_child(folium.Element(legend_html))


m

#m.save("baseline_station_map.html")